In [1]:
from indexify import IndexifyClient, ExtractionGraph

In [2]:
client = IndexifyClient()

In [3]:
extraction_graph_spec = """
name: 'pdflearner'
extraction_policies:
   - extractor: 'tensorlake/paddleocr_extractor'
     name: 'pdf_to_text'
"""

In [4]:
extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)

In [6]:
content_id = client.upload_file("pdflearner", "/home/quamer23nasim38/faster-patient-onboarding-from-medical-history-using-indexify/data/6000267600123615133.pdf")

In [7]:
client.wait_for_extraction(content_id)

Waiting for extraction to complete for content id:  bde2472b2f7a00b2
Extraction completed for content id:  bde2472b2f7a00b2


In [9]:
extracted_content = client.get_extracted_content(ingested_content_id=content_id, graph_name="pdflearner", policy_name="pdf_to_text")

In [10]:
print(extracted_content)

[]
